In [65]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import os.path as path
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from faker import Factory
from scipy import interpolate
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math 
import itertools 
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from ipywidgets import IntSlider, Output
from IPython.display import clear_output
import matplotlib.colors as mcolors
import h5py
import re
import matplotlib
import ipysheet

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables, calibration, ion_selection
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.mc import mc_tools
from pyccapt.calibration.leap_tools import ccapt_tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [67]:
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()
display(tdc)

Dropdown(description='Data mode:', options=('surface_concept', 'roentdec', 'leap_epos', 'leap_pos', 'ato_v6'),…

$$\textbf{You can specify which dataset to use in below block}$$

In [68]:
# exctract needed data from Pandas data frame as an numpy array
# create an instance of the Variables opject
variables = share_variables.Variables()
variables.pulse_mode = pulse_mode
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path 
variables.result_data_name = 'range_data_' + variables.dataset_name
variables.result_path = dataset_main_path + '/' + '/ions_selection/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
    
# Create data farame out of hdf5 file dataset
data = data_tools.load_data(dataset_path, tdc.value, mode='processed')
# extract data from the path and create the Variable object
data_tools.extract_data(data, variables, flightPathLength_d.value, max_mc.value)

The maximum time of flight: 5010


In [69]:
print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')
print('Total number of Ions:', len(data))
data

The data will be saved on the path: D:/pyccapt/tests/data/data_TiO2
The dataset name after saving is: range_data_mc_calibration_cropped_data_TiO2
The figures will be saved on the path: D:/pyccapt/tests/data/data_TiO2//ions_selection/
Total number of Ions: 133227


,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,24.294740,24.223358,5300.0,0.0,0,0.0,1245.142212,3.007860,0.758752,0,0
1,0.0,0.0,0.0,266.730920,267.360874,5300.0,0.0,0,0.0,4044.909180,-0.593483,2.150420,0,0
2,0.0,0.0,0.0,32.997502,32.944901,5300.0,0.0,0,0.0,1441.265747,0.415422,-2.184250,0,0
3,0.0,0.0,0.0,24.347984,24.276713,5300.0,0.0,0,0.0,1251.313354,-0.085065,3.816530,0,0
4,0.0,0.0,0.0,82.639443,82.710638,5300.0,0.0,0,0.0,2267.203369,-0.093658,-2.549270,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133222,0.0,0.0,0.0,94.948381,95.053096,5300.0,0.0,0,0.0,2435.999512,-1.069740,3.071310,0,0
133223,0.0,0.0,0.0,253.682612,254.270044,5300.0,0.0,0,0.0,3931.918457,-0.522718,0.673361,0,0
133224,0.0,0.0,0.0,25.956776,25.888875,5300.0,0.0,0,0.0,1285.479126,2.264250,-1.965060,0,0
133225,0.0,0.0,0.0,162.073257,162.372762,5300.0,0.0,0,0.0,3165.060791,-0.215881,-2.668120,0,0


In [70]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc_c', 'mc_c'), ('tof_c', 'tof_c')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=1), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=fixed(True));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_33476\3981313127.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1),


interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('mc…

In [71]:
bin_size=widgets.FloatText(value=0.1, description='bin size:')
prominence=widgets.IntText(value=50, description='peak prominance:')
distance=widgets.IntText(value=1, description='peak distance:')
lim_tof=widgets.IntText(value=400, description='lim tof/mc:')
percent=widgets.IntText(value=50, description='percent MRP:')
index_fig = widgets.IntText(value=1, description='fig index:')
plot_peak = widgets.Dropdown(
    options=[('True', True), ('False', False)],
    description='plot peak:'
)

def hist_plot_p(variables, plot):
    
    with out:
        clear_output(True)
        # clear the peak_idx
        variables.peaks_idx = []
        bin_size_p = bin_size.value
        prominence_p = prominence.value
        distance_p = distance.value
        lim_tof_p = lim_tof.value
        percent_p = percent.value
        index_fig_p = index_fig.value
        plot_peak_p = plot_peak.value
        tools.hist_plot(variables.mc_c[variables.mc_c < lim_tof_p], variables, bin_size_p, label='mc', range_data=None, adjust_label=False, ranging=False,
              log=True, mode='count', percent=percent_p, peaks_find=True, peaks_find_plot=plot_peak_p, plot=True, prominence=prominence_p,
              distance=None, h_line=False, selector='peak', fast_hist=True, fig_name=index_fig_p, text_loc='right',
              fig_size=(9, 5), background={'calculation': False})
def hist_plot_r(variables, plot):
    
    with out:
        clear_output(True)
        bin_size_r = bin_size.value
        prominence_r = prominence.value
        distance_r = distance.value
        lim_tof_r = lim_tof.value
        percent_r = percent.value
        index_fig_r = index_fig.value
        plot_peak_r = plot_peak.value
        print('=============================')
        print('Press left click to draw a line')
        print('Press right click to remove a line')
        print('Press r to remove all the line')
        print('Press a to automatically draw lines')
        print('Hold shift and use mouse scroll for zooming on x axis')
        print('Hold ctrl and left mouse bottom to move a line')
        print('=============================')
        tools.hist_plot(variables.mc_c[variables.mc_c < lim_tof_r], variables, bin_size_r, label='mc', range_data=None, adjust_label=False, ranging=False,
              log=True, mode='count', percent=percent_r, peaks_find=True, peaks_find_plot=plot_peak_r, plot=True, prominence=prominence_r,
              distance=None, h_line=False, selector='range', fast_hist=True, fig_name=index_fig_r, text_loc='right',
              fig_size=(9, 5), background={'calculation': False})
        
def hist_plot(variables, plot):
    with out:
        clear_output(True)
        bin_size_r = bin_size.value
        prominence_r = prominence.value
        distance_r = distance.value
        lim_tof_r = lim_tof.value
        percent_r = percent.value
        index_fig_r = index_fig.value
        plot_peak_r = plot_peak.value
        tools.hist_plot(variables.mc_c[variables.mc_c < lim_tof_r], variables, bin_size_r, label='mc', range_data=None, adjust_label=False, ranging=False,
              log=True, mode='count', percent=percent_r, peaks_find=True, peaks_find_plot=plot_peak_r, plot=True, prominence=prominence_r,
              distance=None, h_line=True, selector=None, fast_hist=True, fig_name=index_fig_r, text_loc='right',
              fig_size=(9, 5), background={'calculation': False})

In [72]:
# element calculate
peak_val = widgets.FloatText(value=1.1, description='peak value:')
charge = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)
aboundance_threshold = widgets.FloatText(value=0.0, description='threshold aboundance:', min=0, max=1, step=0.1)
num_element = widgets.IntText(value=1, description='num element:')
# formula calculate
formula_m = widgets.Text(
    value='',
    placeholder='Type a formula',
    description='Isotope formula:',
    disabled=False
)

molecule_charge = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)

aboundance_threshold_manual = widgets.FloatText(value=0.0, description='threshold aboundance:', min=0, max=1, step=0.1)

# molecule create
formula_com = widgets.Text(
    value='',
    placeholder="Elements ','",
    description='Elements:',
    disabled=False
)
complexity = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='complexity:'
)

charge_com = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)

aboundance_threshold_com = widgets.FloatText(value=0.0, description='threshold aboundance:', min=0, max=1, step=0.1)



In [73]:
plot_button_p = widgets.Button(
    description='plot hist',
)
plot_button_r = widgets.Button(
    description='plot hist',
)
plot_button = widgets.Button(
    description='plot hist',
)


find_elem_button = widgets.Button(
    description='find element',
)
find_mole_button = widgets.Button(
    description='find molecule',
)

formula_find_button = widgets.Button(
    description='molecule calculate',
)

add_ion_button = widgets.Button(
    description='add ion',
)
romove_ion_button = widgets.Button(
    description='remove ion',
)
show_color = widgets.Button(
    description='show color',
)
change_color = widgets.Button(
    description='change color',
)

color_picker = widgets.ColorPicker(description='Select a color:')
row_index = widgets.IntText(value=0, description='index row:')
mass_unknown = widgets.FloatText(value=0.0, description='unknown mass:')
@plot_button_p.on_click
def plot_on_click_p(b, plot=True):
    hist_plot_p(variables, plot)
@plot_button_r.on_click
def plot_on_click_b(b, plot=True):
    hist_plot_r(variables, plot)
    
@plot_button.on_click
def plot_on_click_b(b, plot=True):
    hist_plot(variables, plot)
    
@find_elem_button.on_click
def vol_on_click(b,):
    with output2:
        peak_val_s = peak_val.value
        charge_s = charge.value
        num_element_s = num_element.value
        aboundance_threshold_s = aboundance_threshold.value
        clear_output(True)
        df = ion_selection.find_close_element(peak_val_s, num_element_s, aboundance_threshold_s, charge=charge_s, variables=variables)
        display(df)
    
@find_mole_button.on_click
def mol_on_click(b,):
    with output2:
        if formula_com.value == '':
            print("Input is empty. Type at least two elements.")
        else:
            df = ion_selection.molecule_create(formula_com.value.split(','), complexity.value, molecule_charge.value, aboundance_threshold_manual.value)
            clear_output(True)
            display(df)
    
@formula_find_button.on_click
def manual_formula(b,):
    with output2:
        if formula_m.value == '':
            print("Input is empty. Type the formula.")
        else:
            df = ion_selection.molecule_isotope_list(formula_m.value, molecule_charge.value, aboundance_threshold_manual.value, latex=True, variables=variables)
            clear_output(True)
            display(df)

@add_ion_button.on_click
def add_ion_to_range_dataset(b,):
    ion_selection.rangging_dataset_create(variables, row_index.value, mass_unknown.value)
    with output3:
        clear_output(True)
        display(variables.range_data)
@romove_ion_button.on_click
def remove_ion_to_range_dataset(b,):
    if len(variables.range_data) >= 1:
        variables.range_data = variables.range_data.drop(len(variables.range_data) - 1)
        with output3:
            clear_output(True)
            display(variables.range_data)
        
@show_color.on_click
def show_color_ions(b,):
    with output3:
        clear_output(True)
        display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))
@change_color.on_click
def change_color_m(b,):
    with output3:
        selected_color = mcolors.to_hex(color_picker.value)
        variables.range_data.at[row_index.value, 'color'] = selected_color
        clear_output(True)
        display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))
        
tab1 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, plot_peak, HBox(children=[plot_button_p])])
tab2 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, plot_peak, HBox(children=[plot_button_r])])
tab3 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, plot_peak, HBox(children=[plot_button])])
tab4 = VBox(children=[HBox(children=[VBox(children=[peak_val, num_element, charge, aboundance_threshold, 
                      find_elem_button]),  VBox(children=[formula_com,complexity,charge_com, aboundance_threshold_com, find_mole_button]),
                      VBox(children=[formula_m, molecule_charge, aboundance_threshold_manual, formula_find_button])]), VBox(children=[row_index, color_picker, mass_unknown, add_ion_button, romove_ion_button, 
                                                                                                                                      show_color, change_color])])



tabs1 = widgets.Tab(children=[tab1, tab2, tab3])
tabs2 = widgets.Tab(children=[tab4])
tabs1.set_title(0, 'peak finder')
tabs1.set_title(1, 'rangging')
tabs1.set_title(2, 'rangged')
tabs2.set_title(0, 'element finder')
# Create two Output widgets to capture the output of each plot
out = Output()
output2 = Output()
output3 = Output()

# Create an HBox to display the buttons side by side
buttons_layout = widgets.HBox([tabs1, tabs2])

# Create a VBox to display the output widgets below the buttons
output_layout_h = widgets.HBox([out, output3])
output_layout = widgets.VBox([output_layout_h, output2])


# Display the buttons and the output widgets
display(buttons_layout, output_layout)


with output3:
    display(variables.range_data)

In [14]:
aa = ion_selection.molecule_create('C,O', 4, charge=4, abundance_threshold=0.0, variables=variables)
display(aa)

TypeError: molecule_create() got an unexpected keyword argument 'variables'

In [49]:
def mc_hist_plot(bin_size, lim_mc, hist_color_range, figname):
    peaks_ini, peaks_y_ini, peak_widths_p_ini, _ = tools.hist_plot(variables.mc_c[variables.mc_c < lim_mc], variables, bin=bin_size, range_data=variables.range_data, ranging=True, hist_color_range=hist_color_range, adjust_label=False, peaks_find=True, peaks_find_plot=True, plot=True, label='mc', fig_name=figname)
   
interact_manual(mc_hist_plot, bin_size=widgets.FloatText(value=0.1), lim_mc=widgets.IntText(value=400), hist_color_range=widgets.Dropdown(options=[('True', True), ('False', False)])
                , figname=widgets.Text(value='ranged'));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_33476\116442335.py:4: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(mc_hist_plot, bin_size=widgets.FloatText(value=0.1), lim_mc=widgets.IntText(value=400), hist_color_range=widgets.Dropdown(options=[('True', True), ('False', False)])


interactive(children=(FloatText(value=0.1, description='bin_size'), IntText(value=400, description='lim_mc'), …

In [50]:
display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

,ion,mass,mc,mc_low,mc_up,color,peak_count,element,complex,isotope,charge
0,${}^{1}H^{+}$,1.010000,1.581404,1.357690,1.886822,#b2aa2d,163.000000,H,1,1,+
1,${}^{27}Al^{2+}$,13.490000,13.505377,12.998589,14.321418,#e7e0d1,64219.000000,Al,1,27,2+
2,${}^{27}Al^{+}$,26.980000,26.993149,25.856491,28.660889,#e7e0d1,589415.000000,Al,1,27,+


In [51]:
variables.range_data.dtypes

ion            object
mass          float64
mc            float64
mc_low        float64
mc_up         float64
color          object
peak_count    float64
element        object
complex         int64
isotope         int64
charge         object
dtype: object

In [54]:
# save the new data
name_save_file = variables.result_data_path + '/' + variables.result_data_name + '.h5'
data_tools.store_df_to_hdf(variables.range_data,  'df', name_save_file)

In [55]:
# save data in csv format
name_save_file = variables.result_data_path + '/' + variables.result_data_name + '.csv'
data_tools.store_df_to_csv(variables.range_data, name_save_file)